
## 0a. Install and running with Jupyter and Apache Toree

do the following steps

#### Download and Install JDK

wget -c --header "Cookie: oraclelicense=accept-securebackup-cookie" http://download.oracle.com/otn-pub/java/jdk/8u144-b01/090f390dda5b47b9b721c7dfaa008135/jdk-8u144-linux-x64.rpm
yum localinstall jdk-8u121-linux-x64.rpm

#### Download and Install Spark

    wget -c https://d3kbcqa49mib13.cloudfront.net/spark-2.2.0-bin-hadoop2.7.tgz
    mkdir ~/spark
    tar -zxvf spark-2.2.0-bin-hadoop2.6.tgz -C ~/spark/

#### Set Environment variables

Add following lines in your .bash_profile or .bashrc

export SPARK_HOME=~/spark/spark-2.2.0-bin-hadoop2.7
export PATH=$PATH:$SPARK_HOME/bin

#### Test Spark Installation

You can start Spark shell and Python shell by typing below commands

#To start spark shell
spark-shell
#To start python shell
pyspark

#### Download and Install Anaconda

    wget -c https://repo.continuum.io/archive/Anaconda3-5.0.0.1-Linux-x86_64.sh
    bash Anaconda3-5.0.0.1-Linux-x86_64.sh

####  Download and Install Apache Toree

    wget -c https://dist.apache.org/repos/dist/dev/incubator/toree/0.2.0/snapshots/dev1/toree-pip/toree-0.2.0.dev1.tar.gz
    pip install toree-0.2.0.dev1.tar.gz
    jupyter toree install --spark_home=$SPARK_HOME --interpreters=Scala,PySpark,SQL --user
#### Start Jupyter Server ( for local machine)

    jupyter notebook --no-browser



In [1]:
sc.version

2.3.0

In [2]:
#No need to create new spark sesion since it is created by he notebook
#from pyspark.sql import SparkSession
#
#spark = SparkSession \
#    .builder \
#    .appName("Python Spark SQL basic example") \
#    .config("spark.some.config.option", "some-value") \
#    .getOrCreate()
spark    

# Kredi Risk Değerlendirme (Veri Analizi ve Makine Öğrenimi)
## Amaç ve Kapsam
Bu çalışma kapsamında banka  müşterilerinin kredi riskini analiz edeceğiz. Bir müşterinin kişisel bilgileri ile geçmiş banka kayıtları verilerini birleştirerek kredi borcunu zamanında kapatıp kapatması arasındaki ilişkiyi analiz etmeye çalışacağız.
Açık kaynaklı veriler kullanacağız

 ## Verilerin Toplanması
Yeh ve Lien (2009) tarafından kredi riskini değerlendirmek amacıyla kullanılan bir veri setini http://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients paylaşmış durumdadır. Tayvan daki banka müşterilerinin kredi risk değerlenmesi üzerine yapılan biir araştırmanın parçası olarak yayınlanmıştır.

Veriler etiketlidir , bu yüzden güdümlü öğrenme teknikleri kullanılacaktır.

* Güdümlü öğrenme kullanarak ve müşterilerin 6 aylık örüntülerine bakarak kredi ödemesinde sıkıntı olacak müşterileri tahmin edeceğiz.
* Veri setini inceleyerek yeni öznitelikler türeteceğiz.
* Kullandığımız yöntemlerin başarısını inceleyeceğiz.


```bash
mkdir ./veri/
wget http://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls -O ./veri/default-of-credit-card-clients.xls 

wget https://safirdepo.b3lab.org/shares/public/share/ZRIm14IrTlOK35cE1C9Yc7N52RFMyeuf/Credit_Scoring_test.csv -O ./veri/Credit_Scoring_test.csv

# Eğer veriyi HDFS e aktaracaksak aşağıdaki komutlar çalıştırılır

hdfs dfs -put ./veri/Credit_Scoring_train.csv /tmp/
hdfs dfs -put ./veri/Credit_Scoring_test.csv /tmp/
hdfs dfs -ls /tmp
```




In [5]:
#HDFS
trainfilehdfs = "/tmp/Credit_Scoring_train.csv"
testfilehdfs = "/tmp/Credit_Scoring_test.csv"

#Lokal veri
trainfile = "./veri/Credit_Scoring_train.csv"
testfile = "./veri/Credit_Scoring_test.csv"

train_df = spark.read.csv(trainfile,header='true',sep=";")
test_df = spark.read.csv(testfile,header='true',sep=";")

Yüklediğimiz verisetlerini inceleyelim


In [7]:
train_df.show(5)

+------------+---------+------+----------+--------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+----------------+-------+
| CUSTOMER_ID|LIMIT_BAL|   SEX| EDUCATION|MARRIAGE|AGE|PAYMENT_1|PAYMENT_2|PAYMENT_3|PAYMENT_4|PAYMENT_5|PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|PAYMENT_AMOUNT_1|PAYMENT_AMOUNT_2|PAYMENT_AMOUNT_3|PAYMENT_AMOUNT_4|PAYMENT_AMOUNT_5|PAYMENT_AMOUNT_6|default|
+------------+---------+------+----------+--------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+------

In [8]:
test_df.show(5)

+------------+---------+------+----------+--------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+----------------+-------+
| CUSTOMER_ID|LIMIT_BAL|   SEX| EDUCATION|MARRIAGE|AGE|PAYMENT_1|PAYMENT_2|PAYMENT_3|PAYMENT_4|PAYMENT_5|PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|PAYMENT_AMOUNT_1|PAYMENT_AMOUNT_2|PAYMENT_AMOUNT_3|PAYMENT_AMOUNT_4|PAYMENT_AMOUNT_5|PAYMENT_AMOUNT_6|default|
+------------+---------+------+----------+--------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+------

In [9]:
features = train_df.columns
features

['CUSTOMER_ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAYMENT_1', 'PAYMENT_2', 'PAYMENT_3', 'PAYMENT_4', 'PAYMENT_5', 'PAYMENT_6', 'REMAINING_AMOUNT_1', 'REMAINING_AMOUNT_2', 'REMAINING_AMOUNT_3', 'REMAINING_AMOUNT_4', 'REMAINING_AMOUNT_5', 'REMAINING_AMOUNT_6', 'PAYMENT_AMOUNT_1', 'PAYMENT_AMOUNT_2', 'PAYMENT_AMOUNT_3', 'PAYMENT_AMOUNT_4', 'PAYMENT_AMOUNT_5', 'PAYMENT_AMOUNT_6', 'default']

In [10]:
print(train_df.count(), test_df.count())

(19800, 10200)


## Veri Sözlüğü

Veride aşağıdaki kolonlar yer almaktadır:

* CUSTOMER_ID: Müşteriye özel ID.
* LIMIT_BAL: Kredi limiti.
* SEX: Cinsiyet (Male, Female). 
* EDUCATION: Eğitim (Gradschool, University, Highschool). 
* MARRIAGE: Medeni durum (Married, Single). 
* AGE: Yaş (year). 
* PAYMENT_1 - PAYMENT_6: Ay bazında geçmiş ödemelerdeki gecikme (1 - Eylül 2005, 6 - Nisan 2005 olacak şekilde).
* REMAINING_AMOUNT_1 - REMAINING_AMOUNT_6: Kalan borç miktarı (1 - Eylül 2005, 6 - Nisan 2005 olacak şekilde).
* PAYMENT_AMOUNT_1 - PAYMENT_AMOUNT_6: Önceki ödemeler (1 - Eylül 2005, 6 - Nisan 2005 olacak şekilde).
* default: Çıktı değeri, borcun zamanında ödenip ödenmediği (0 = Zamanında, 1 = Geç)


Önce sayısal kolonlara bakarak eksik değer olup olmadığına bakalım.

## Veri Tiplerinin düzenlenmesi / dönüştürülmesi

Sayısal olan verilerin eğer değilse uygun tiplere dönüştürülmesi gereklidir.
Aşağıda rasyonel sayılar, csv vb dosyalarda  karakter dizini olarak yazıldıklarında, okunduktan sonra da karakter dizininden rasyonel sayılara dönüştürülmeleri gerekir.
Bazı okuyuculara hangi kolonların dönüştürülmeis gerektiği belirtildiğinde, otomatk dönüşüm yapar. bu örnekte kendimiz dönüşüm yapacağız.

In [11]:
train_df.schema

StructType(List(StructField(CUSTOMER_ID,StringType,true),StructField(LIMIT_BAL,StringType,true),StructField(SEX,StringType,true),StructField(EDUCATION,StringType,true),StructField(MARRIAGE,StringType,true),StructField(AGE,StringType,true),StructField(PAYMENT_1,StringType,true),StructField(PAYMENT_2,StringType,true),StructField(PAYMENT_3,StringType,true),StructField(PAYMENT_4,StringType,true),StructField(PAYMENT_5,StringType,true),StructField(PAYMENT_6,StringType,true),StructField(REMAINING_AMOUNT_1,StringType,true),StructField(REMAINING_AMOUNT_2,StringType,true),StructField(REMAINING_AMOUNT_3,StringType,true),StructField(REMAINING_AMOUNT_4,StringType,true),StructField(REMAINING_AMOUNT_5,StringType,true),StructField(REMAINING_AMOUNT_6,StringType,true),StructField(PAYMENT_AMOUNT_1,StringType,true),StructField(PAYMENT_AMOUNT_2,StringType,true),StructField(PAYMENT_AMOUNT_3,StringType,true),StructField(PAYMENT_AMOUNT_4,StringType,true),StructField(PAYMENT_AMOUNT_5,StringType,true),StructFie

In [12]:
from pyspark.sql.functions import udf
from pyspark.sql.functions import col , column
from pyspark.sql.types import DoubleType
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType 

toIntegerfunc = udf( lambda x: int(x) if x else int(0), IntegerType())
toFloatfunc = udf( lambda x: float(x) if x else float(0), FloatType())
toStringfunc = udf( lambda x: x if x else "other", StringType())


train_df = train_df.withColumn("LIMIT_BAL",toFloatfunc(train_df['LIMIT_BAL']))
train_df = train_df.withColumn("AGE",toIntegerfunc(train_df['AGE']))

train_df = train_df.withColumn("PAYMENT_1",toFloatfunc(train_df['PAYMENT_1']))
train_df = train_df.withColumn("PAYMENT_2",toFloatfunc(train_df['PAYMENT_2']))
train_df = train_df.withColumn("PAYMENT_3",toFloatfunc(train_df['PAYMENT_3']))
train_df = train_df.withColumn("PAYMENT_4",toFloatfunc(train_df['PAYMENT_4']))
train_df = train_df.withColumn("PAYMENT_5",toFloatfunc(train_df['PAYMENT_5']))
train_df = train_df.withColumn("PAYMENT_6",toFloatfunc(train_df['PAYMENT_6']))

train_df = train_df.withColumn("REMAINING_AMOUNT_1",toFloatfunc(train_df['REMAINING_AMOUNT_1']))
train_df = train_df.withColumn("REMAINING_AMOUNT_2",toFloatfunc(train_df['REMAINING_AMOUNT_2']))
train_df = train_df.withColumn("REMAINING_AMOUNT_3",toFloatfunc(train_df['REMAINING_AMOUNT_3']))
train_df = train_df.withColumn("REMAINING_AMOUNT_4",toFloatfunc(train_df['REMAINING_AMOUNT_4']))
train_df = train_df.withColumn("REMAINING_AMOUNT_5",toFloatfunc(train_df['REMAINING_AMOUNT_5']))
train_df = train_df.withColumn("REMAINING_AMOUNT_6",toFloatfunc(train_df['REMAINING_AMOUNT_6']))

train_df = train_df.withColumn("PAYMENT_AMOUNT_1",toFloatfunc(train_df['PAYMENT_AMOUNT_1']))
train_df = train_df.withColumn("PAYMENT_AMOUNT_2",toFloatfunc(train_df['PAYMENT_AMOUNT_2']))
train_df = train_df.withColumn("PAYMENT_AMOUNT_3",toFloatfunc(train_df['PAYMENT_AMOUNT_3']))
train_df = train_df.withColumn("PAYMENT_AMOUNT_4",toFloatfunc(train_df['PAYMENT_AMOUNT_4']))
train_df = train_df.withColumn("PAYMENT_AMOUNT_5",toFloatfunc(train_df['PAYMENT_AMOUNT_5']))
train_df = train_df.withColumn("PAYMENT_AMOUNT_6",toFloatfunc(train_df['PAYMENT_AMOUNT_6']))

train_df = train_df.withColumn("default",toIntegerfunc(train_df['default']))


In [13]:
train_df.schema

StructType(List(StructField(CUSTOMER_ID,StringType,true),StructField(LIMIT_BAL,FloatType,true),StructField(SEX,StringType,true),StructField(EDUCATION,StringType,true),StructField(MARRIAGE,StringType,true),StructField(AGE,IntegerType,true),StructField(PAYMENT_1,FloatType,true),StructField(PAYMENT_2,FloatType,true),StructField(PAYMENT_3,FloatType,true),StructField(PAYMENT_4,FloatType,true),StructField(PAYMENT_5,FloatType,true),StructField(PAYMENT_6,FloatType,true),StructField(REMAINING_AMOUNT_1,FloatType,true),StructField(REMAINING_AMOUNT_2,FloatType,true),StructField(REMAINING_AMOUNT_3,FloatType,true),StructField(REMAINING_AMOUNT_4,FloatType,true),StructField(REMAINING_AMOUNT_5,FloatType,true),StructField(REMAINING_AMOUNT_6,FloatType,true),StructField(PAYMENT_AMOUNT_1,FloatType,true),StructField(PAYMENT_AMOUNT_2,FloatType,true),StructField(PAYMENT_AMOUNT_3,FloatType,true),StructField(PAYMENT_AMOUNT_4,FloatType,true),StructField(PAYMENT_AMOUNT_5,FloatType,true),StructField(PAYMENT_AMOUNT

In [14]:
train_df.describe().show()

+-------+--------------------+------------------+------+----------+--------+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|summary|         CUSTOMER_ID|         LIMIT_BAL|   SEX| EDUCATION|MARRIAGE|               AGE|          PAYMENT_1|           PAYMENT_2|           PAYMENT_3|           PAYMENT_4|           PAYMENT_5|           PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|  PAYMENT_AMOUNT_1|  PAYMENT_AMOUNT_2|  PAYMENT_AMOUNT_3|  PAYMENT_AMOUNT_4|  PAYMENT_AMOUNT_5|  PAYMENT_AMOUNT_6|            default|
+-------+--------------------+------------------+------+------

In [15]:
train_df.describe(['LIMIT_BAL']).show()
train_df.describe(['AGE']).show()

+-------+------------------+
|summary|         LIMIT_BAL|
+-------+------------------+
|  count|             19800|
|   mean| 167938.7717171717|
| stddev|129545.90887361384|
|    min|               0.0|
|    max|         1000000.0|
+-------+------------------+

+-------+------------------+
|summary|               AGE|
+-------+------------------+
|  count|             19800|
|   mean|35.505959595959595|
| stddev| 9.281760782566714|
|    min|                 0|
|    max|                79|
+-------+------------------+



In [16]:
train_df.describe([i for i in train_df.columns if 'REMAINING_AMOUNT' in i]).show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|             19800|             19800|             19800|             19800|             19800|             19800|
|   mean| 51265.96813131313| 49428.55823232323|  47197.1903030303| 43617.47111111111| 40696.43348484849|  39142.8997979798|
| stddev| 73465.13784487464| 71323.10512233786| 69425.73609526413| 64693.95917192512| 61500.58081796579| 60140.54539011229|
|    min|         -165580.0|          -33350.0|         -157264.0|         -170000.0|          -81334.0|         -339603.0|
|    max|          964511.0|          983931.0|         1664089.0|          891586.0|          927171.0|          961664.0|
+-------

In [17]:
train_df.describe([i for i in train_df.columns if 'PAYMENT' in i]).show()

+-------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|          PAYMENT_1|           PAYMENT_2|           PAYMENT_3|           PAYMENT_4|           PAYMENT_5|           PAYMENT_6|  PAYMENT_AMOUNT_1|  PAYMENT_AMOUNT_2|  PAYMENT_AMOUNT_3|  PAYMENT_AMOUNT_4|  PAYMENT_AMOUNT_5|  PAYMENT_AMOUNT_6|
+-------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|              19800|               19800|               19800|               19800|               19800|               19800|             19800|             19800|             19800|             19800|             19800|             19800|
|   

In [18]:
train_df.createOrReplaceTempView("traindatatable")

In [65]:
qdf = spark.sql("select LIMIT_BAL,count(*) as cnt  from traindatatable group by LIMIT_BAL order by LIMIT_BAL")
#qdf = spark.sql("select LIMIT_BAL,count(*) as cnt  from traindatatable group by LIMIT_BAL")

In [54]:
qdf.show()

+---------+----+
|LIMIT_BAL| cnt|
+---------+----+
|      0.0|   3|
|  10000.0| 315|
|  20000.0|1327|
|  30000.0|1058|
|  40000.0| 138|
|  50000.0|2176|
|  60000.0| 546|
|  70000.0| 484|
|  80000.0|1009|
|  90000.0| 441|
| 100000.0| 704|
| 110000.0| 382|
| 120000.0| 485|
| 130000.0| 500|
| 140000.0| 486|
| 150000.0| 720|
| 160000.0| 451|
| 170000.0| 374|
| 180000.0| 669|
| 190000.0| 151|
+---------+----+
only showing top 20 rows



In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pdf = qdf.toPandas()
pdf.head()

   LIMIT_BAL   cnt
0        0.0     3
1    10000.0   315
2    20000.0  1327
3    30000.0  1058
4    40000.0   138

In [57]:
#%matplotlib inline
#q1 = pdf.set_index('cnt')
pdf.plot.bar()
plt.show()

In [62]:
pdf.plot.scatter(x='LIMIT_BAL', y='cnt', s=4);
plt.show()

In [66]:
qdf = spark.sql("select AGE,count(*) from traindatatable group by AGE")

In [70]:
pdf = qdf.toPandas()
pdf.plot.scatter(x='AGE',y='count(1)',s=5)
plt.show()

In [72]:
qdf  = spark.sql("select AGE,EDUCATION,count(*)   from traindatatable group by AGE, EDUCATION")

In [73]:
qdf = spark.sql("select AGE,EDUCATION,MARRIAGE, count(*)  from traindatatable where default=1 group by AGE, EDUCATION,MARRIAGE")

In [74]:
qdf = spark.sql("select AGE,EDUCATION,MARRIAGE, count(*)  from traindatatable where default=1 group by AGE, EDUCATION,MARRIAGE")

## Veri Temizleme / Dönüştürme
Veriyi incelerken fark edilen durumları düzeltmek için bazı işlemler yapacağız.

In [75]:
train_df = train_df.filter(train_df['LIMIT_BAL']>0)
train_df.describe(['LIMIT_BAL']).show()

+-------+------------------+
|summary|         LIMIT_BAL|
+-------+------------------+
|  count|             19797|
|   mean|167964.22084154165|
| stddev|129539.22575911172|
|    min|           10000.0|
|    max|         1000000.0|
+-------+------------------+



In [76]:
#AGE kolonunu median değeriyle dolduruyoruz. Missing değer ve strategy (nasıl dolduracağımız) bilgisini verelim.

median_dict = dict()
median_dict['AGE'] = train_df.stat.approxQuantile('AGE', [0.5], 0.001)[0]
print(median_dict)
train_df = train_df.na.fill(median_dict)

{'AGE': 34.0}


In [77]:
#  null(olmayan)  değerlerler var mı ?
train_df.describe(['SEX']).show()
train_df.describe(['EDUCATION']).show()
train_df.describe(['MARRIAGE']).show()

+-------+------+
|summary|   SEX|
+-------+------+
|  count| 19797|
|   mean|  null|
| stddev|  null|
|    min|Female|
|    max|  Male|
+-------+------+

+-------+----------+
|summary| EDUCATION|
+-------+----------+
|  count|     19517|
|   mean|      null|
| stddev|      null|
|    min|Gradschool|
|    max|University|
+-------+----------+

+-------+--------+
|summary|MARRIAGE|
+-------+--------+
|  count|   19555|
|   mean|    null|
| stddev|    null|
|    min| Married|
|    max|  Single|
+-------+--------+



In [78]:
# null değerler için yeni bir değer yazalım
na_dict = dict()
na_dict['EDUCATION'] = 'MISSING_EDU'
na_dict['MARRIAGE'] = 'MISSING_MAR'

train_df = train_df.fillna(na_dict)


In [79]:
train_df.describe(['SEX']).show()
train_df.describe(['EDUCATION']).show()
train_df.describe(['MARRIAGE']).show()

+-------+------+
|summary|   SEX|
+-------+------+
|  count| 19797|
|   mean|  null|
| stddev|  null|
|    min|Female|
|    max|  Male|
+-------+------+

+-------+----------+
|summary| EDUCATION|
+-------+----------+
|  count|     19797|
|   mean|      null|
| stddev|      null|
|    min|Gradschool|
|    max|University|
+-------+----------+

+-------+-----------+
|summary|   MARRIAGE|
+-------+-----------+
|  count|      19797|
|   mean|       null|
| stddev|       null|
|    min|MISSING_MAR|
|    max|     Single|
+-------+-----------+



In [80]:
train_df.show(3)

+------------+---------+------+-----------+--------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+----------------+-------+
| CUSTOMER_ID|LIMIT_BAL|   SEX|  EDUCATION|MARRIAGE|AGE|PAYMENT_1|PAYMENT_2|PAYMENT_3|PAYMENT_4|PAYMENT_5|PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|PAYMENT_AMOUNT_1|PAYMENT_AMOUNT_2|PAYMENT_AMOUNT_3|PAYMENT_AMOUNT_4|PAYMENT_AMOUNT_5|PAYMENT_AMOUNT_6|default|
+------------+---------+------+-----------+--------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+---

In [81]:
train_df.createOrReplaceTempView("traindatatable2")

## Veri değerlendirme (tekrar)


In [82]:
qdf = spark.sql("select * from traindatatable2")

In [83]:
qdf.toPandas().plot.hist()
plt.show()

## Makine Öğrenimi ile Risk Değerlendirmesi
Müşterilerin zamanında mı ödeme yapıp yapmayacaklarını tahmin etmeye çalışalım

In [84]:
# Hangi kolonlar kullanılacak

categorical_columns = ['SEX', 'EDUCATION', 'MARRIAGE' ]
numerical_columns = ['LIMIT_BAL', 'AGE', 'PAYMENT_1', 'PAYMENT_2', 'PAYMENT_3', 'PAYMENT_4', 'PAYMENT_5', 'PAYMENT_6', 'REMAINING_AMOUNT_1', 'REMAINING_AMOUNT_2', 'REMAINING_AMOUNT_3', 'REMAINING_AMOUNT_4', 'REMAINING_AMOUNT_5', 'REMAINING_AMOUNT_6', 'PAYMENT_AMOUNT_1', 'PAYMENT_AMOUNT_2', 'PAYMENT_AMOUNT_3', 'PAYMENT_AMOUNT_4', 'PAYMENT_AMOUNT_5', 'PAYMENT_AMOUNT_6']

# default kolonu, ismini label olarak değiştirelim
train_df = train_df.withColumnRenamed('default', 'label')

dftrain = train_df.select(categorical_columns + numerical_columns + ['label'] )
dftrain.show(5)

+------+-----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+----------------+-----+
|   SEX|  EDUCATION|MARRIAGE|LIMIT_BAL|AGE|PAYMENT_1|PAYMENT_2|PAYMENT_3|PAYMENT_4|PAYMENT_5|PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|PAYMENT_AMOUNT_1|PAYMENT_AMOUNT_2|PAYMENT_AMOUNT_3|PAYMENT_AMOUNT_4|PAYMENT_AMOUNT_5|PAYMENT_AMOUNT_6|label|
+------+-----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+----------------+-----+
|  Male|MISSING_EDU| M



Son olarak yeni öznitelikler türetelim. 

* Müşterilerin kredi limitleri oldukça önemli bir bilgi gibi duruyor. Büyük olasılıkla yaşlandıkça limitiniz de artacak.  'LIMIT_PER_AGE' adlı bir kolonla limiti yaşa bölerek yeni bir değişken oluşturalım. Bu sayede aynı limite sahip iki müşteriden daha genç olanını ayırdedebileceğimiz bir değişken oluşturuyoruz. Genç ve yüksek limitli bir müşteri daha cazip bir müşteri olabilir.
* Aynı zamanda bankaya olan borcunuzla maddi durumunuzun da ilişkisini kurmak yararlı olabilir. Aynı borca sahip iki müşteriden limiti daha yüksek olan daha az riskli bir müşteri olabilir. REMAINING_AMOUNT_1 kolonunu LIMIT_BAL kolununa bölerek yeni bir değişken türetelim.


In [85]:
MaletoBinaryfunc = udf( lambda x: int(1) if 'Male' in x else int(0) )
FemaletoBinaryfunc = udf( lambda x: int(1) if 'Female' in x else int(0) )

#Yeni değişkenler oluşturuyoruz.
dftrain= dftrain.withColumn('LIMIT_PER_AGE', dftrain['LIMIT_BAL'] / dftrain['AGE'])
dftrain= dftrain.withColumn('RELATIVE_REMAINING_AMOUNT',  dftrain['REMAINING_AMOUNT_1'] / dftrain['LIMIT_BAL'])
#dftrain = dftrain.withColumn('MALE', MaletoBinaryfunc(dftrain['SEX']))
#dftrain = dftrain.withColumn('FEMALE', FemaletoBinaryfunc(dftrain['SEX']))

numerical_columns.append('LIMIT_PER_AGE')
numerical_columns.append('RELATIVE_REMAINING_AMOUNT')
#if 'SEX' in categorical_columns:
#    categorical_columns.remove('SEX')
#    categorical_columns.append('MALE')
#    categorical_columns.append('FEMALE')

print(categorical_columns)

print(numerical_columns)

['SEX', 'EDUCATION', 'MARRIAGE']
['LIMIT_BAL', 'AGE', 'PAYMENT_1', 'PAYMENT_2', 'PAYMENT_3', 'PAYMENT_4', 'PAYMENT_5', 'PAYMENT_6', 'REMAINING_AMOUNT_1', 'REMAINING_AMOUNT_2', 'REMAINING_AMOUNT_3', 'REMAINING_AMOUNT_4', 'REMAINING_AMOUNT_5', 'REMAINING_AMOUNT_6', 'PAYMENT_AMOUNT_1', 'PAYMENT_AMOUNT_2', 'PAYMENT_AMOUNT_3', 'PAYMENT_AMOUNT_4', 'PAYMENT_AMOUNT_5', 'PAYMENT_AMOUNT_6', 'LIMIT_PER_AGE', 'RELATIVE_REMAINING_AMOUNT']


In [86]:
dftrain.schema

StructType(List(StructField(SEX,StringType,true),StructField(EDUCATION,StringType,false),StructField(MARRIAGE,StringType,false),StructField(LIMIT_BAL,FloatType,true),StructField(AGE,IntegerType,true),StructField(PAYMENT_1,FloatType,true),StructField(PAYMENT_2,FloatType,true),StructField(PAYMENT_3,FloatType,true),StructField(PAYMENT_4,FloatType,true),StructField(PAYMENT_5,FloatType,true),StructField(PAYMENT_6,FloatType,true),StructField(REMAINING_AMOUNT_1,FloatType,true),StructField(REMAINING_AMOUNT_2,FloatType,true),StructField(REMAINING_AMOUNT_3,FloatType,true),StructField(REMAINING_AMOUNT_4,FloatType,true),StructField(REMAINING_AMOUNT_5,FloatType,true),StructField(REMAINING_AMOUNT_6,FloatType,true),StructField(PAYMENT_AMOUNT_1,FloatType,true),StructField(PAYMENT_AMOUNT_2,FloatType,true),StructField(PAYMENT_AMOUNT_3,FloatType,true),StructField(PAYMENT_AMOUNT_4,FloatType,true),StructField(PAYMENT_AMOUNT_5,FloatType,true),StructField(PAYMENT_AMOUNT_6,FloatType,true),StructField(label,In

In [87]:
dftrain.show()

+------+-----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+----------------+-----+------------------+-------------------------+
|   SEX|  EDUCATION|MARRIAGE|LIMIT_BAL|AGE|PAYMENT_1|PAYMENT_2|PAYMENT_3|PAYMENT_4|PAYMENT_5|PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|PAYMENT_AMOUNT_1|PAYMENT_AMOUNT_2|PAYMENT_AMOUNT_3|PAYMENT_AMOUNT_4|PAYMENT_AMOUNT_5|PAYMENT_AMOUNT_6|label|     LIMIT_PER_AGE|RELATIVE_REMAINING_AMOUNT|
+------+-----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+-------

In [88]:
dftrain.describe().show()

+-------+------+----------+-----------+------------------+-----------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-----------------+-------------------------+
|summary|   SEX| EDUCATION|   MARRIAGE|         LIMIT_BAL|              AGE|          PAYMENT_1|           PAYMENT_2|           PAYMENT_3|           PAYMENT_4|           PAYMENT_5|           PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|  PAYMENT_AMOUNT_1|  PAYMENT_AMOUNT_2|  PAYMENT_AMOUNT_3|  PAYMENT_AMOUNT_4|  PAYMENT_AMOUNT_5|  PAYMENT_AMOUNT_6|              label|    LIMIT_PER_AGE|RELATIVE_REMAINING_AMOUNT|
+-------+---


### Pipeline 
Kategorik ve sayısal özelliklerin modele uygun veritiplerine dönüştürülmesi için bir pipeline oluşturulur.
Bir pipeline (boru hattı), genellikle bir dizi **transformer** ve **estimator** aşamasından sonra bir DataFrame hazırlayan işlemlerden oluşur.
Ardından modelleme ve ardından tahmini bir modeli eğitme işlemeleri yapılır.

- **StringIndexer** kategorik özellikleri dizinlere dönüştüren bir dönüştürücü
- **VectorAssembler** Kategorik özellikleri tek bir vektöre birleştiren bir birleştirici
- **VectorIndexer** Bir kategorik özellik vektörü için dizinler oluşturan dönüştürücü
- **VectorAssembler**, sürekli sayısal özellikler içeren bir vektör oluşturan birleştirici
- **MinMaxScaler** Sürekli sayısal özellikleri normalleştiren bir dönüştürücü
- **VectorAssembler** Kategorik ve sürekli özellikler içeren bir vektör oluşturan bir birleştirici
- Sınıflandırma modelini eğiten bir **DecisionTreeClassifier**.

### Sınıflandırma 
Önce karar ağacı ile sınıflandırma yapacağız. Bu sınıflandırmayı pipeline bir parçası olarak uygulayacağız



In [89]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler


In [90]:
dftrain.show(2)

+------+-----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+----------------+-----+-----------------+-------------------------+
|   SEX|  EDUCATION|MARRIAGE|LIMIT_BAL|AGE|PAYMENT_1|PAYMENT_2|PAYMENT_3|PAYMENT_4|PAYMENT_5|PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|PAYMENT_AMOUNT_1|PAYMENT_AMOUNT_2|PAYMENT_AMOUNT_3|PAYMENT_AMOUNT_4|PAYMENT_AMOUNT_5|PAYMENT_AMOUNT_6|label|    LIMIT_PER_AGE|RELATIVE_REMAINING_AMOUNT|
+------+-----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+---------

In [91]:

#kategorik özellikleri indeksliyoruz
edustrIdx  = StringIndexer(inputCol='EDUCATION', outputCol= 'EDUCATIONIdx') 
marstrIdx  = StringIndexer(inputCol='MARRIAGE', outputCol= 'MARRIAGEIdx') 
sexstrIdx  = StringIndexer(inputCol='SEX', outputCol= 'SEXIdx') 

#indexleri vector haline getiriyoruz
catVect = VectorAssembler(inputCols =['EDUCATIONIdx', 'MARRIAGEIdx', 'SEXIdx' ]  , outputCol="catFeatures")

#kategorik veri içeren vektorü indeksliyoruz
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures")

#sayısal veri içeren özellikleri vektör haline getiriyoruz
numVect = VectorAssembler(inputCols = ['LIMIT_BAL','PAYMENT_1','REMAINING_AMOUNT_1', 'PAYMENT_AMOUNT_1' ], outputCol="numFeatures")

#sayısal veriler üzerinde normallizasyon işlemi ile değerleri belirlenen bir aralığa dönüştürüyoruz
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")

#kategorik ve sayısal verileri birleştiren bir vektör oluşturuyoruz
featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")

dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")



In [92]:
all_stages = [edustrIdx, marstrIdx, sexstrIdx, catVect, catIdx, numVect, minMax, featVect, dt]

pipeline = Pipeline(stages=all_stages)

pipelineModel = pipeline.fit(dftrain)

print "Pipeline complete!"

Pipeline complete!


## Test verisi dönüştürme

Eğitim/Train setinde yaptığımız dönüşümleri test seti üzerinde de yapacağız.

In [93]:
# Aynı işlemleri test üzerinde de yapıyoruz

from pyspark.sql.functions import udf
from pyspark.sql.functions import col , column
from pyspark.sql.types import DoubleType 
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType 

toIntegerfunc = udf( lambda x: int(x) if x else int(0), IntegerType())
toDoublefunc = udf( lambda x: float(x) if x else float(0), DoubleType())
toStringfunc = udf( lambda x: x if x else "other", StringType())


test_df = spark.read.csv(testfile,header='true',sep=";")

test_df = test_df.withColumn("LIMIT_BAL",toIntegerfunc(test_df['LIMIT_BAL']))
test_df = test_df.withColumn("AGE",toIntegerfunc(test_df['AGE']))

test_df = test_df.withColumn("PAYMENT_1",toIntegerfunc(test_df['PAYMENT_1']))
test_df = test_df.withColumn("PAYMENT_2",toIntegerfunc(test_df['PAYMENT_2']))
test_df = test_df.withColumn("PAYMENT_3",toIntegerfunc(test_df['PAYMENT_3']))
test_df = test_df.withColumn("PAYMENT_4",toIntegerfunc(test_df['PAYMENT_4']))
test_df = test_df.withColumn("PAYMENT_5",toIntegerfunc(test_df['PAYMENT_5']))
test_df = test_df.withColumn("PAYMENT_6",toIntegerfunc(test_df['PAYMENT_6']))

test_df = test_df.withColumn("REMAINING_AMOUNT_1",toIntegerfunc(test_df['REMAINING_AMOUNT_1']))
test_df = test_df.withColumn("REMAINING_AMOUNT_2",toIntegerfunc(test_df['REMAINING_AMOUNT_2']))
test_df = test_df.withColumn("REMAINING_AMOUNT_3",toIntegerfunc(test_df['REMAINING_AMOUNT_3']))
test_df = test_df.withColumn("REMAINING_AMOUNT_4",toIntegerfunc(test_df['REMAINING_AMOUNT_4']))
test_df = test_df.withColumn("REMAINING_AMOUNT_5",toIntegerfunc(test_df['REMAINING_AMOUNT_5']))
test_df = test_df.withColumn("REMAINING_AMOUNT_6",toIntegerfunc(test_df['REMAINING_AMOUNT_6']))

test_df = test_df.withColumn("PAYMENT_AMOUNT_1",toIntegerfunc(test_df['PAYMENT_AMOUNT_1']))
test_df = test_df.withColumn("PAYMENT_AMOUNT_2",toIntegerfunc(test_df['PAYMENT_AMOUNT_2']))
test_df = test_df.withColumn("PAYMENT_AMOUNT_3",toIntegerfunc(test_df['PAYMENT_AMOUNT_3']))
test_df = test_df.withColumn("PAYMENT_AMOUNT_4",toIntegerfunc(test_df['PAYMENT_AMOUNT_4']))
test_df = test_df.withColumn("PAYMENT_AMOUNT_5",toIntegerfunc(test_df['PAYMENT_AMOUNT_5']))
test_df = test_df.withColumn("PAYMENT_AMOUNT_6",toIntegerfunc(test_df['PAYMENT_AMOUNT_6']))

test_df = test_df.withColumn("default",toIntegerfunc(test_df['default']))

test_df = test_df.filter(test_df['LIMIT_BAL']>0)

median_dict = dict()
median_dict['AGE'] = test_df.stat.approxQuantile('AGE', [0.5], 0.001)[0]
test_df = test_df.na.fill(median_dict)

test_df = test_df.fillna(na_dict)

test_df= test_df.withColumn('LIMIT_PER_AGE', test_df['LIMIT_BAL'] / test_df['AGE'])
test_df= test_df.withColumn('RELATIVE_REMAINING_AMOUNT',  test_df['REMAINING_AMOUNT_1'] / test_df['LIMIT_BAL'])

test_df= test_df.withColumn('trueLabel', test_df['default'])

dftest = test_df.select(categorical_columns + numerical_columns + ['trueLabel'])

dftest.show(3)

+------+-----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+----------------+------------------+-------------------------+---------+
|   SEX|  EDUCATION|MARRIAGE|LIMIT_BAL|AGE|PAYMENT_1|PAYMENT_2|PAYMENT_3|PAYMENT_4|PAYMENT_5|PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|PAYMENT_AMOUNT_1|PAYMENT_AMOUNT_2|PAYMENT_AMOUNT_3|PAYMENT_AMOUNT_4|PAYMENT_AMOUNT_5|PAYMENT_AMOUNT_6|     LIMIT_PER_AGE|RELATIVE_REMAINING_AMOUNT|trueLabel|
+------+-----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------

## Test Verisinde Tahmin Yapılması
Eğitim seti ile oluşturduğumuz pipeline dan test setini geçiriyoruz ve tahmini sonuçlar ile gerçek sonuçları karşılaştırıyoruz.

In [94]:
prediction = pipelineModel.transform(dftest)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+------------------------------------------------------------------------------+----------+---------+
|features                                                                      |prediction|trueLabel|
+------------------------------------------------------------------------------+----------+---------+
|[3.0,0.0,1.0,0.0,0.1111111111111111,0.15444596939538496,0.0022712871287128714]|0.0       |0        |
|[1.0,1.0,0.0,0.0,0.1111111111111111,0.15324429625578825,0.005940594059405941] |0.0       |0        |
|[1.0,1.0,0.0,0.0,0.1111111111111111,0.15328057651994398,0.002499009900990099] |0.0       |0        |
|[1.0,1.0,1.0,0.0,0.5555555555555556,0.1524868351309762,0.0]                   |1.0       |0        |
|(7,[0,5],[1.0,0.14696073148091615])                                           |1.0       |1        |
|[1.0,0.0,0.0,0.0,0.3333333333333333,0.14953928488944696,0.0039603960396039604]|1.0       |0        |
|[1.0,0.0,0.0,0.0,0.1111111111111111,0.15156920991318398,0.0022237623762376237]|0.

## Tahmin performansının ölçümü

Test verisini üzerinde tahminimizin ne kadar doğru olduğunu ölçüyoruz. **Accuracy** , **Precision** ve **Recall** değerleri hedeflerdiğimiz değerlere ulaşıp ulaşmadığını ölçüyoruz.

In [95]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Accuracy", (tp + tn) / (tp+fp+tn+fn) ),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])
metrics.show()



+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|             793.0|
|       FP|             359.0|
|       TN|            7542.0|
|       FN|            1506.0|
| Accuracy| 0.817156862745098|
|Precision|0.6883680555555556|
|   Recall|0.3449325793823401|
+---------+------------------+



Tahmin değerlerinin  ham halleri ve hesaplanan olasılık değerleri de incelenebilir.


In [96]:
prediction.select("rawPrediction", "probability", "prediction", "trueLabel").show(100, truncate=False)

+--------------+----------------------------------------+----------+---------+
|rawPrediction |probability                             |prediction|trueLabel|
+--------------+----------------------------------------+----------+---------+
|[1107.0,293.0]|[0.7907142857142857,0.2092857142857143] |0.0       |0        |
|[650.0,182.0] |[0.78125,0.21875]                       |0.0       |0        |
|[1107.0,293.0]|[0.7907142857142857,0.2092857142857143] |0.0       |0        |
|[139.0,458.0] |[0.23283082077051925,0.7671691792294807]|1.0       |0        |
|[5.0,8.0]     |[0.38461538461538464,0.6153846153846154]|1.0       |1        |
|[139.0,458.0] |[0.23283082077051925,0.7671691792294807]|1.0       |0        |
|[1107.0,293.0]|[0.7907142857142857,0.2092857142857143] |0.0       |0        |
|[139.0,458.0] |[0.23283082077051925,0.7671691792294807]|1.0       |0        |
|[650.0,182.0] |[0.78125,0.21875]                       |0.0       |0        |
|[82.0,109.0]  |[0.4293193717277487,0.57068062827225

**ROC**  metriğinde alanın ne kadar olduğu hesaplanabilir. ***Area Under ROC*** ( **AUR** ) değerleri farklı modeller için karşılaştırıldığında, AUR büyük olan model daha başarılıdır.


In [98]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur = evaluator.evaluate(prediction)
print "AUR = ", aur


AUR =  0.505820423786


In [102]:
# prototip tahminleyici modelimizi kaydedelim.
model_path = "/tmp/"
dtc_path = model_path + 'decisiontree.model'
dt.save(dtc_path)


In [103]:
test0 = spark.createDataFrame([('Male','Gradschool', 'Single', 10000,44, 0, 0, 0, 0, 0, 0, 8958, 8774, 9461, 9457, 9650, 7500, 1147, 1224, 1000, 193, 150, 0,227.27272727272728, 0.8958)], 
   ["SEX", "EDUCATION","MARRIAGE","LIMIT_BAL", "AGE","PAYMENT_1","PAYMENT_2","PAYMENT_3","PAYMENT_4","PAYMENT_5","PAYMENT_6","REMAINING_AMOUNT_1","REMAINING_AMOUNT_2","REMAINING_AMOUNT_3","REMAINING_AMOUNT_4","REMAINING_AMOUNT_5","REMAINING_AMOUNT_6","PAYMENT_AMOUNT_1","PAYMENT_AMOUNT_2","PAYMENT_AMOUNT_3","PAYMENT_AMOUNT_4","PAYMENT_AMOUNT_5","PAYMENT_AMOUNT_6", "LIMIT_PER_AGE","RELATIVE_REMAINING_AMOUNT"])

test0.show()

prediction = pipelineModel.transform(test0)
prediction.select("rawPrediction", "probability", "prediction").show()

+----+----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+----------------+------------------+-------------------------+
| SEX| EDUCATION|MARRIAGE|LIMIT_BAL|AGE|PAYMENT_1|PAYMENT_2|PAYMENT_3|PAYMENT_4|PAYMENT_5|PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|PAYMENT_AMOUNT_1|PAYMENT_AMOUNT_2|PAYMENT_AMOUNT_3|PAYMENT_AMOUNT_4|PAYMENT_AMOUNT_5|PAYMENT_AMOUNT_6|     LIMIT_PER_AGE|RELATIVE_REMAINING_AMOUNT|
+----+----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+-----------

Perfromans metriği ölçümü için daha önce yaptığımız işlemleri bir fonksiyonda toparlayalım.

In [105]:
def getMetrics(prediction):
    tp = float(prediction.filter("prediction == 1.0 AND truelabel == 1").count())
    fp = float(prediction.filter("prediction == 1.0 AND truelabel == 0").count())
    tn = float(prediction.filter("prediction == 0.0 AND truelabel == 0").count())
    fn = float(prediction.filter("prediction == 0.0 AND truelabel == 1").count())
    
    metrics = spark.createDataFrame([
     ("TP", tp),
     ("FP", fp),
     ("TN", tn),
     ("FN", fn),
     ("Accuracy", (tp + tn) / (tp+fp+tn+fn) ),
     ("Precision", tp / (tp + fp)),
     ("Recall", tp / (tp + fn))],["metric", "value"])
     
    return metrics

## Model / Pipeline İyileştirme 

Modeli, pipeline ı değiştirerek , daha iyi bir  tahmin performansı elde edebilir miyiz ?

In [106]:
#Kategorik özelikleri daha önceki şekilde yapacağımız için değitrmiyoruz
#kategorik özellikleri indeksliyoruz
#edustrIdx  = StringIndexer(inputCol='EDUCATION', outputCol= 'EDUCATIONIdx') 
#marstrIdx  = StringIndexer(inputCol='MARRIAGE', outputCol= 'MARRIAGEIdx') 
#sexstrIdx  = StringIndexer(inputCol='SEX', outputCol= 'SEXIdx') 

#indexleri vector haline getiriyoruz
#catVect = VectorAssembler(inputCols =['EDUCATIONIdx', 'MARRIAGEIdx', 'SEXIdx' ]  , outputCol="catFeatures")

#kategorik veri içeren vektorü indeksliyoruz
#catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures")

#sayısal veri içeren özellikleri vektör haline getiriyoruz, bu  özelikleri genişletip daraltabiliri, (feature engineering)
allnumcols = ['LIMIT_BAL', 'AGE', 'PAYMENT_1', 'PAYMENT_2', 'PAYMENT_3', 'PAYMENT_4', 'PAYMENT_5', 'PAYMENT_6']

#['REMAINING_AMOUNT_1', 'REMAINING_AMOUNT_2', 'REMAINING_AMOUNT_3', 'REMAINING_AMOUNT_4', 'REMAINING_AMOUNT_5', 'REMAINING_AMOUNT_6', 'PAYMENT_AMOUNT_1', 'PAYMENT_AMOUNT_2','PAYMENT_AMOUNT_3', 'PAYMENT_AMOUNT_4', 'PAYMENT_AMOUNT_5', 'PAYMENT_AMOUNT_6'] 
#['LIMIT_PER_AGE', 'RELATIVE_REMAINING_AMOUNT']

numVect2 = VectorAssembler(inputCols = allnumcols, outputCol="numFeatures")

#sayısal veriler üzerinde normallizasyon işlemi ile değerleri belirlenen bir aralığa dönüştürüyoruz
minMax2 = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")

#kategorik ve sayısal verileri birleştiren bir vektör oluşturuyoruz
featVect2 = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")

#Pipeline da sadece dönüştürme işlemleri olacak
all_transform_stages = [edustrIdx, marstrIdx, sexstrIdx, catVect, catIdx, numVect2, minMax2, featVect2]

pipeline_transform = Pipeline(stages=all_transform_stages)



In [107]:
#Eğitim kümesini dönüştürelim
dftrain_transform = pipeline_transform.fit(dftrain).transform(dftrain)

#Dönüştürülmüş özelliklerin isimlerini ve değerlerini inceleyelim
dftrain_transform.show(2)
#Pipeline sonucunda oluşan özellikleri inceleyelim.
dftrain_transform.select(['EDUCATIONIdx','MARRIAGEIdx','SEXIdx', 'catFeatures', 'idxCatFeatures', 'numFeatures', 'normFeatures', 'features','label']).show(5)


+------+-----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+----------------+-----+-----------------+-------------------------+------------+-----------+------+-------------+--------------+--------------------+--------------------+--------------------+
|   SEX|  EDUCATION|MARRIAGE|LIMIT_BAL|AGE|PAYMENT_1|PAYMENT_2|PAYMENT_3|PAYMENT_4|PAYMENT_5|PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|PAYMENT_AMOUNT_1|PAYMENT_AMOUNT_2|PAYMENT_AMOUNT_3|PAYMENT_AMOUNT_4|PAYMENT_AMOUNT_5|PAYMENT_AMOUNT_6|label|    LIMIT_PER_AGE|RELATIVE_REMAINING_AMOUNT|EDUCATIONIdx|MARRIAGEIdx|SEXIdx|  catFeatures|idxCatFeatures|         numFeatures|        normFeatures|            features|
+------+----

In [108]:
#test verisini de aynı pipeline ile dönüştürelim
dftest_transform = Pipeline(stages=all_transform_stages).fit(dftest).transform(dftest)
dftest_transform.show(2)

+------+-----------+--------+---------+---+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+----------------+----------------+------------------+-------------------------+---------+------------+-----------+------+-------------+--------------+--------------------+--------------------+--------------------+
|   SEX|  EDUCATION|MARRIAGE|LIMIT_BAL|AGE|PAYMENT_1|PAYMENT_2|PAYMENT_3|PAYMENT_4|PAYMENT_5|PAYMENT_6|REMAINING_AMOUNT_1|REMAINING_AMOUNT_2|REMAINING_AMOUNT_3|REMAINING_AMOUNT_4|REMAINING_AMOUNT_5|REMAINING_AMOUNT_6|PAYMENT_AMOUNT_1|PAYMENT_AMOUNT_2|PAYMENT_AMOUNT_3|PAYMENT_AMOUNT_4|PAYMENT_AMOUNT_5|PAYMENT_AMOUNT_6|     LIMIT_PER_AGE|RELATIVE_REMAINING_AMOUNT|trueLabel|EDUCATIONIdx|MARRIAGEIdx|SEXIdx|  catFeatures|idxCatFeatures|         numFeatures|        normFeatures|            features|
+-

## Karar Ağacı ile Sınıflandırma

Ayrıntılı bilgi için
http://spark.apache.org/docs/2.1.0/api/python/pyspark.ml.html?highlight=decisiontreeclassifier#pyspark.ml.classification.DecisionTreeClassifier


In [109]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier( labelCol="label", featuresCol="features")

model = dt.fit(dftrain_transform)

prediction = model.transform(dftest_transform)

getMetrics(prediction).show()

+---------+-------------------+
|   metric|              value|
+---------+-------------------+
|       TP|              827.0|
|       FP|              390.0|
|       TN|             7511.0|
|       FN|             1472.0|
| Accuracy| 0.8174509803921569|
|Precision| 0.6795398520953163|
|   Recall|0.35972161809482384|
+---------+-------------------+



Kullandığımız Karar ağacının özelliklerini inceleyelim

In [110]:
print 'impurity ', dt.getImpurity()
print 'Model depth', model.depth
print 'Model nodes', model.numNodes
print 'Feature Importances', model.featureImportances
print 'Number of features', model.numFeatures

impurity  gini
Model depth 5
Model nodes 59
Feature Importances (11,[0,3,4,5,6,7,8,9,10],[0.0065278466102,0.0352388490962,0.00536355126512,0.732617936613,0.141709543019,0.00207917227261,0.0391378700089,0.0282183799397,0.00910685117576])
Number of features 11


In [111]:
print(model.toDebugString)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4ec797d9732afed652f4) of depth 5 with 59 nodes
  If (feature 5 <= 0.2777777777777778)
   If (feature 6 <= 0.3125)
    If (feature 8 <= 0.3125)
     If (feature 5 <= 0.16666666666666666)
      If (feature 3 <= 0.05555555555555555)
       Predict: 0.0
      Else (feature 3 > 0.05555555555555555)
       Predict: 0.0
     Else (feature 5 > 0.16666666666666666)
      If (feature 10 <= 0.0625)
       Predict: 0.0
      Else (feature 10 > 0.0625)
       Predict: 0.0
    Else (feature 8 > 0.3125)
     If (feature 9 <= 0.25)
      If (feature 3 <= 0.17676767676767677)
       Predict: 0.0
      Else (feature 3 > 0.17676767676767677)
       Predict: 0.0
     Else (feature 9 > 0.25)
      If (feature 3 <= 0.11616161616161616)
       Predict: 0.0
      Else (feature 3 > 0.11616161616161616)
       Predict: 0.0
   Else (feature 6 > 0.3125)
    If (feature 9 <= 0.25)
     If (feature 3 <= 0.03535353535353536)
      If (feature 10 <= 0.4375)


### SORU - 1
Karar ağacında maxDepth parametresini 30 olan bir model oluşturun ve performansını ölçün


In [ ]:

print(".....")
dt = DecisionTreeClassifier( ..... )

model = dt.fit(.....)

prediction = model.transform(....)

getMetrics(prediction).show()

### SORU - 2
Daha fazla özellik seçerek modeli eğitin ve perfromansını ölçün


In [ ]:
allnumcols = [....]

numVect3 = VectorAssembler(inputCols = allnumcols, outputCol="numFeatures")

#sayısal veriler üzerinde normallizasyon işlemi ile değerleri belirlenen bir aralığa dönüştürüyoruz
minMax3 = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")

#kategorik ve sayısal verileri birleştiren bir vektör oluşturuyoruz
featVect3 = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")

#Pipeline da sadece dönüştürme işlemleri olacak
soru_transform_stages = [....]

soru_pipeline_transform = Pipeline(stages=......)

#Eğitim kümesini dönüştürelim
soru_dftrain_transform = soru_pipeline_transform.fit(...).transform(....)

soru_dftest_transform = Pipeline(stages=....).fit(...).transform(...)

dt = DecisionTreeClassifier( ..... )

model = dt.fit(.....)

prediction = model.transform(....)

getMetrics(prediction).show()



## Lojistik Regresyon ile Sınıflandırma
Ayrıntılı bilgi için 
http://spark.apache.org/docs/2.1.0/api/python/pyspark.ml.html?highlight=logisticregression#pyspark.ml.classification.LogisticRegression


In [113]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)

model = lr.fit(dftrain_transform)

prediction = model.transform(dftest_transform)

getMetrics(prediction).show()

+---------+--------------------+
|   metric|               value|
+---------+--------------------+
|       TP|               111.0|
|       FP|                47.0|
|       TN|              7854.0|
|       FN|              2188.0|
| Accuracy|  0.7808823529411765|
|Precision|  0.7025316455696202|
|   Recall|0.048281861678990866|
+---------+--------------------+



Parametreleri değiştirerek modeli eğitelim. Threshold değerini 0.35 olarak verelim.

In [114]:
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3, threshold=0.35)

model = lr.fit(dftrain_transform)

prediction = model.transform(dftest_transform)

getMetrics(prediction).show()

+---------+-------------------+
|   metric|              value|
+---------+-------------------+
|       TP|              552.0|
|       FP|              271.0|
|       TN|             7630.0|
|       FN|             1747.0|
| Accuracy|  0.802156862745098|
|Precision| 0.6707168894289186|
|   Recall|0.24010439321444105|
+---------+-------------------+



### SORU- 3

soru - 2 de kullandığnız özelikleri (feature vector) lojistik regresyon sınıflandırıcısı kullanarak model , oluşturup perfromansını ökçünüz.


In [ ]:
lr = LogisticRegression( ..... )

model = lr.fit(.....)

prediction = model.transform(....)

getMetrics(prediction).show()



### SORU - 4 
Bu çalışma nın sonunda elde ettiğiniz bilgileri bir rapor/sunum haline getiriniz. Ana başlıkları aşağıdaki gibi kullanabilirsiniz.




## < Raporun Başlığı >

### Özet
Analiz kapsamı çalışması kapsamında çözüm bulmaya çalıştığınız probleminizi, sorularınızı  ve bulgularınızı 1 kısa paragrafta özetleyin (en fazla 4-6 cümle). Özet kısmında hangi veri setini, hangi soruyu, hangi yöntemi kullandığını ve bulguları kısaca yazınız.

### Motivasyon
Veriyle çözmek istediğiniz problemi açıklayınız. Hedefiniz, kitlenizin çözmeye çalıştığınız proje / sorunla ilgilenmesini sağlamaktır. Keşfedilen sorunun ve nedeninin (ve kime) anliz çalışmasının değerli olduğunu belirtmeniz gerekir.

### Veri Seti
Veri setini burada açıklayınız. Veri kümesindeki hangi verilerin olduğunu, ne kadar verinin olduğunu ve veri kümesini nerede bulduğunuzu (varsa) yazınız.

### Veri Hazırlama ve Temizleme
Verileri analiz için hazırlamak için ne yapmanız gerekiyor? Varsa, hangi veri kümesiyle karşılaştığınız sorunları tanımlayınız.

### Araştırma Konusu
Veri setini kullanarak cevaplamayı amaçladığınız araştırma sorunuz nedir? Araştırma sorununun iyi tanımlanmış olduğundan emin olunuz (ayrıntılar için proje açıklamasına bakın).

### Yöntem
Verileri analiz etmek için hangi yöntemleri kullandınız ve neden bunları uygun buldunuz? Yöntemlerinizi  yeterli derecede (kısa olmak kayyadıyla) tarif ediniz.

### Bulgular
Bulgularınızı anlatın. Raporunuza en az bir görselleştirme ekleyiniz (daha fazla bilgi ekleyebilirsiniz). Genel bir izleyici için görselleştirme doğru, erişilebilir ve zarif olmalıdır.
Kesin bir sonuca varmanız gerekmese de bulgularınızın araştırma sorunuzla nasıl ilişkili olduğunu anlatmalısınız.

### Kısıtlar
Mümkünse, bulgularınıza kısıtlarınızı yazınız. Örneğin, bu sonuçların Tayvan banka müşterileriiçin geçerli olduğunu, ancak banka  yapılarındaki farklılıklardan dolayı diğer bankalara uygulanamayabilir olduğunu unutmayınız.

### Değerlendirmeler
Genel sonuçlarınızı ve değerlendirmelerinizi yazınız.

### Teşekkür
Verilerinizi nereden buldunuz? Çalışmalarınızı bilgilendirmek için diğer gayrı resmi analiz kullandınız mı? Arkadaşlarınız veya meslektaşlarınız tarafından çalışmalarınız hakkında geribildirim aldınız mı? 

### Referanslar
Mümkünse, çalışmalarınızda kullandığınız referansları yazınız.